# High Frequency Trading - Avellaneda-Stoikov Model

---

## Introduction

Avellaneda & Stoikov (2006)'s seminal paper *High-frequency trading in a limit order book* develops an **optimal (stock) market-making strategy** for HFT in a stochastic limit order book. A dealer dynamically posts bid/ask quotes to capture spreads while controlling inventory risk through continuous-time optimal stochastic control.

Traditional optimization frameworks address **two core risks**:

* **Inventory risk**: Uncertainty in the value of dealer holdings $q_t$. Under diffusive mid-price dynamics $dS_u = \sigma dW_u$, the terminal wealth $X_T + q_T S_T$ exhibits **quadratic variance** $\text{Var}(q_T S_T) = q_T^2 \sigma^2 T$, which escalates rapidly with $|q_t|$, where $X_T$ is cash earned from realised spreads.
* **Asymmetric information risk** (adverse selection): Informed traders exploit stale quotes, capturing the economic spread while the dealer earns only microstructure edge.

The paper primarily tackles **inventory risk optimization** via a (asymptotically) tractable Hamilton-Jacobi-Bellman PDE, deriving inventory-aware reservation prices $r(t,q_t)$ and optimal spreads that naturally mean-revert positions toward zero.

## Model Setup & Assumptions

### State Variables
$$
\mathbb{S} := (t, q_t, X_t, S_t)
$$ 
where:
* $t$ represents the current time
* $q_t$ represents the current inventory position
* $X_t$ represents the current cash incurred from the realised spreads
* $S_t$ is the mid-price of the stock

  
### Price Process
The stock price process is simply modelled as:

$$
dS_u = \sigma dW_u
$$

Intitutively, a pure Brownian motion with no drift assumes the mid-price follows a diffusive process over the ultra-short HFT horizon (seconds to minutes). Mean-reversion or trends (drifts) operate on longer timescales, so they could be considered less relevant here. 
This diffusion is mathematically essential: combined with CARA utility, it yields normally distributed terminal wealth, enabling a clean closed-form solution for the reservation price $r(t,q)$. 
The authors further note that a Geometric Brownian Motion model (with no drift) is permissible, but runs into problems with mathematical infinities. The results do remain the largely the same, but for simplicity of this text we omit further mention of this below.


### Orders
Model an agent who is going to quote limit orders (i.e. market make) via limit orders:
* Bid price $p^b = s - \delta^b$
* Ask price $p^a = s + \delta^a$

for some $\delta^a$ and $\delta^b$ which need to be calibrated. Intitutively these can be thought of as the distances from the mid price to determine the priority of execution of limit orders.

Assuming a large market buy order of size $Q$ and price $p^Q$, the top $Q$ limit best asks will be executed. The authors define this market impact of this as:
$$
\Delta p := p^Q - s
$$
In particular if a limit order with price $p$ has  $\delta^a < \Delta p$, it will be executed.

The model chosen to lift limit orders via market orders is a Poisson process:
$$
\lambda^a(\delta^a), \quad \lambda^b(\delta^b) 
$$

The authors justify this by affirming that the further away from the mid-price the agent positions his quotes, the less often he will receive buy and sell orders. It is also backed up from econophysics literature who find that:
* The density of market order size is $f^Q(x) \propto x^{-1-a}$
* Market impact follows $\Delta \propto \ln(Q)$

Using this we can derive the intensity $\lambda(\delta)$:
$$
\begin{aligned}
\lambda(\delta) &= \Lambda P(\Delta p > \delta) \\
&= \Lambda P(\ln(Q) > K\delta) \\
&= \Lambda P(Q > e^{K\delta}) \\
&= \Lambda \int_{e^{K\delta}}^\infty x^{-1-\alpha} \, dx \\
&= A e^{-k\delta}
\end{aligned}
$$
fro some constants $A$, $k$ to be calibrated.

### Wealth Process
Given the stochastic choice of market orders we now have wealth given by a jump process:
$$
dX_t = p^a \, dN_t^a - p^b \, dN_t^b
$$
where $N_t^b$ are the stocks bought by the agent, and $N_t^b$ stocks sold by the agent.


### Inventory Process
Similarly now the inventory process is
$$
q_t = N_t^b - N_t^a
$$


### Utility
CARA (Constant Absolute Risk Aversion) utility enables tractable solutions to the otherwise unsolvable 4D Hamilton-Jacobi-Bellman (HJB) equation in market making models. This choice is reasonable for multiple reasons:

$$
u(t, x_t, q_t, S_t) = \mathbb{E}[-\exp(-\gamma(x_t + q_t S_T)) \, | \,  \mathcal{F}_t]
$$


# Model Summary

$$
\begin{cases}
dX_t = \delta^b_t \, dN^b_t - \delta^a_t \, dN^a_t \\
dq_t = dN^b_t - dN^a_t \\
dS_t = \sigma S_t \, dW_t
\end{cases}
$$
where $N^{a,b}_t$ are independent Poisson processes with state-dependent intensities:
$$
\lambda^b_t = A e^{-k \delta^b_t}, \quad \lambda^a_t = A e^{-k \delta^a_t}
$$

We wish to maximise the agent utility given an optimal stochastic control, i.e.
$$
u(s,x,q,t) = \sup_{\delta^a,\delta^b \geq 0} \mathbb{E} \left[ -\exp\left(-\gamma (X_T + q_T S_T)\right) \right | \mathcal{F}_t]
$$

# CARA Utility under Frozen Inventory

In it's current form the CARA utility is a 4D function (with 3 stochastic dimensions). It will unlikely to be untractable. We first consider an agent with "frozen inventory", where the agent holds their inventory of stocks (of size $q$) till terminal time $T$. 

We can now derive a closed form for the CARA utility under this assumption:

Given $S_T | S_t = s \sim \mathcal{N}(s, \sigma^2 \tau)$ where $\tau = T-t$, we derive the moment generating function $M_t(\theta) = \mathbb{E}[e^{\theta S_T} | S_t = s]$:
$$
M_t(\theta) = \int_{-\infty}^{\infty} e^{\theta z} \cdot \frac{1}{\sqrt{2\pi\sigma^2\tau}} \exp\left( -\frac{(z-s)^2}{2\sigma^2\tau} \right) dz
$$
Completing the square of the exponent:
$$
\begin{aligned}
\theta z - \frac{z^2 - 2sz + s^2}{2\sigma^2\tau} &= -\frac{1}{2\sigma^2\tau} \left[ z^2 - 2(s + \theta\sigma^2\tau)z \right] + \frac{s^2}{2\sigma^2\tau} \\
&= -\frac{(z - (s + \theta\sigma^2\tau))^2}{2\sigma^2\tau} + \underbrace{s\theta + \frac{1}{2}\theta^2\sigma^2\tau}_{\text{bonus term}}
\end{aligned}
$$
Recall the integral over the shifted Gaussian is 1, so:
$$
M_t(\theta) = \exp\left( s\theta + \frac{1}{2}\theta^2\sigma^2\tau \right)
$$
Plugging in the CARA condition $\theta = -\gamma q$ gives:
$$
u(t,x,q,s) = -\exp(-\gamma x) \exp(-\gamma q s) \exp\left( \frac{\gamma^2 q^2 \sigma^2}{2} (T-t) \right)
$$

## Indifference Prices

The central innovation of the paper revolves around combinining **indifference prices** and the microstructure of the order book.

### Reervation Bid Price

The **reservation bid** price $r^b(t,q_t)$ is the certain **buy price** making the dealer indifferent between:

1. Buying 1 share immediately at $r^b(t,q_t)$:
   $$
   u\big(t, x - r^b(t,q_t), q_t + 1, S_t\big)
   $$

2. **Continuing optimal market-making** from current state:
   $$
   u\big(t, x, q_t, S_t\big)
   $$

Indifference here means equality:
$$
u\big(t, x - r^b(t, q), q + 1, s\big) = u\big(t, x, q, s\big)
$$

Now substituting this back into CARA utility, we have
$$
r^b(t, q) = s + (- 1 - 2q) \frac{\gamma \sigma^2 (T - t)}{2}
$$

### Reservation Ask Price
and a similar argument for the ask prices gives the **reservation ask** price:
$$
r^a(t, q) = s + (1 - 2q) \frac{\gamma \sigma^2 (T - t)}{2}
$$

### Indifference Price
Finally, define the  as the average of these two prices:
$$
r(t, q) = s - q \gamma \sigma^2 (T - t)
$$

The interpretation is simple:
* If the agent is long stock ($q > 0$), the indifference price is below mid-price, indicating an intention to liquidate.
* Conversely if the agent is short stock ($q < 0$), the indifference price is above mid-price, since the agent is willing to purchase at a higher price.

## Derivation of the HJB PDE

By the dynamic programming theorem:
$$
u(t,x,q,s) = \sup_{\delta_{[t,t+h]}} \mathbb{E}^\delta_t \Big[ u(t+h, X^\delta_{t+h}, q^\delta_{t+h}, S_{t+h}) \Big]
$$

By the Jump-Itô formula we have:
$$du_r = \mathcal{L}^0 u_r \, dr + \sigma u_s \, dW_r + [u(+)-u] dN^b_r + [u(-)-u] dN^a_r$$
where $\mathcal{L}^0 u = \partial_t u + \frac{1}{2}\sigma^2 u_{ss}$. 

Decompose the raw Poissons such that we expose the drift: $dN^i_r = d\tilde{N}^i_r + \lambda^i(\delta^i) \, dr$:
$$du_r = \Big\{ \mathcal{L}^0 u_r + \lambda^b[u(+)-u] + \lambda^a[u(-)-u] \Big\} dr$$
$$+ \sigma s_r u_s \, dW_r + [u(+)-u] d\tilde{N}^b_r + [u(-)-u] d\tilde{N}^a_r$$

Such that we have the full generator
$$
\mathcal{L}^\delta u := \mathcal{L}^0 u + \lambda^b[u(+)-u] + \lambda^a[u(-)-u]
$$

Integrate over $[t,t+h]$:
$$
u(t+h,X_{t+h},q_{t+h},s_{t+h}) = u(t,x,q,s) + \int_t^{t+h} \mathcal{L}^\delta u_r \, dr
$$
$$
+ \int_t^{t+h} \sigma s_r u_s \, dW_r + \int_t^{t+h} [u(+)-u] \, d\tilde{N}^b_r + \int_t^{t+h} [u(-)-u] \, d\tilde{N}^a_r
$$

Take conditional expectation $\mathbb{E}^\delta[\cdot | \mathcal{F}_t]$ on the integrated Itô equation:
$$
\mathbb{E}^\delta \Big[ u(t+h,X_{t+h},q_{t+h},s_{t+h}) \Big| \mathcal{F}_t \Big] = u(t,x,q,s) + \mathbb{E}^\delta \Big[ \int_t^{t+h} \mathcal{L}^\delta u_r \, dr \Big| \mathcal{F}_t \Big]
$$
because martingale integrals vanish:
$$
\begin{cases}
\mathbb{E}^\delta \left[\int_t^{t+h} u_s \, dW_r\right] = 0 \\
\mathbb{E}^\delta \left[\int_t^{t+h} [u(+)-u] d\tilde{N}^b_r\right] = 0 \\
\mathbb{E}^\delta \left[\int_t^{t+h} [u(-)-u] d\tilde{N}^a_r\right] = 0
\end{cases}
$$


Reintroducing DPP:
$$
\begin{aligned}
u(t,x,q,s) &= u(t,x,q,s) + \mathbb{E}^\delta \Big[ \int_t^{t+h} \mathcal{L}^\delta u_r \, dr \Big| \mathcal{F}_t \Big] \\
0 &= \Big[ \int_t^{t+h} \mathcal{L}^\delta u_r \, dr \Big| \mathcal{F}_t \Big]
\end{aligned}
$$

Which gives us the final HJB PDE with terminal condition:
$$
\begin{cases}
0 = \partial_t u + \frac{1}{2} \sigma^2 u_{ss} + \sup_{\delta^a,\delta^b \geq 0} \Big\{ 
\lambda^b(\delta^b) \big[ u(t, x - (s - \delta^b), q+1, s) - u(t,x,q,s) \big] + 
\lambda^a(\delta^a) \big[ u(t, x + s + \delta^a, q-1, s) - u(t,x,q,s) \big]
\big\} \\
u(T,x,q,s) = -\exp(-\gamma(x + q s))
\end{cases}

## Simplification via Ansatz

Taking the ansatz
$$
u(t,x,q,s) = -\exp\left( -\gamma x \right) \exp\left( -\gamma \theta(t,q,s) \right)
$$

We have via the chain rule:
$$
\begin{cases}
\partial_t u = \gamma \partial_t \theta \, u \\
u_s = \gamma \theta_s u \\
u_{ss} = \gamma \theta_{ss} u + \gamma \theta_s (\gamma \theta_s u) = \gamma \theta_{ss} u + \gamma^2 \theta_s^2 u

\end{cases}
$$

And:
$$
\begin{cases}
u(t, x + (s + \delta^a), q-1, s) - u = u \left[ \exp\left( \gamma (s + \delta^a) + \gamma \left[ \theta(t,q-1,s) - \theta(t,q,s) \right] \right) - 1 \right] \\
u(t, x - (s - \delta^b), q+1, s) - u = u \left[ \exp\left( \gamma (s - \delta^b) + \gamma \left[ \theta(t,q+1,s) - \theta(t,q,s) \right] \right) - 1 \right]
\end{cases}
$$|

Substituting back into the the HJB and dividing by $u$ (which we know < $0$ as $\exp(\cdot) > 0$):
$$
0 = \partial_t \theta + \frac{1}{2} \sigma^2 \partial_{ss} \theta + \frac{1}{2} \gamma \sigma^2 \theta_s^2 
+ \sup_{\delta^a,\delta^b \geq 0} \Big\{ 
\lambda^a(\delta^a) \left[ \exp\left( \gamma (s + \delta^a) + \gamma \left[ \theta(t,q-1,s) - \theta(t,q,s) \right] \right) - 1 \right]
+ 
\lambda^b(\delta^b) \left[ \exp\left( \gamma (s - \delta^b) + \gamma \left[ \theta(t,q+1,s) - \theta(t,q,s) \right] \right) - 1 \right]
\Big\}
$$

Now recall the reservation bid price is defined by
$$
u(t, x, q, s) = u(t, x - r^b, q+1, s)
$$

Similarly substituting the ansatz:
$$
\begin{aligned}
-\exp(-\gamma x) \exp(-\gamma \theta(t,s,q)) &= -\exp(-\gamma (x - r^b)) \exp(-\gamma \theta(t,s,q+1)) \\
r^b(t,s,q) &= \theta(t,s,q+1) - \theta(t,s,q)
\end{aligned}
$$

A similar argument gives the reservation ask price:
$$
\begin{aligned}
r^a(t,s,q) &= \theta(t,s,q) - \theta(t,s,q-1)
\end{aligned}
$$

Which further reduces the HJB PDE nicely to:
$$
\begin{cases}
0 = \partial_t \theta + \frac{1}{2} \sigma^2 \partial_{ss} \theta - \frac{1}{2} \gamma \sigma^2 \theta_s^2 
+ \sup_{\delta^a,\delta^b \geq 0} \Big\{ 
\lambda^a(\delta^a) \left[ \exp\big(\gamma (s + \delta^a - r^a)\big) - 1 \right] + 
\lambda^b(\delta^b) \left[ \exp\big(\gamma (s - \delta^b - r^b)\big) - 1 \right]
\Big\} \\
\theta(s, q, T) = qs 
\end{cases}
$$


## Implicit Optimal Controls

We can find optimal $\delta^a$ and $\delta^b$ via the first order conditions.

### Bid Side FOC 
$$
\begin{aligned}
0 &= \frac{\partial\mathcal{L}}{\partial\delta^b}  \\
&= \lambda^{b}(\delta^b)[e^{\gamma(s-\delta^b-r^b)}-1] - \gamma\lambda^b(\delta^b)e^{\gamma(s-\delta^b-r^b)}
\end{aligned}
$$

Let $z^* := e^{\gamma(s-\delta^b_*-r^b)}$ and we have:
$$
\lambda^{b\prime}(z^*-1) = \gamma\lambda^b z^* \implies z^* = \frac{\lambda^{b\prime}(\delta^b_*)}{\lambda^{b\prime}(\delta^b_*) - \gamma\lambda^b(\delta^b_*)}
$$

Take logs and rearrange and we have:
$$
\boxed{s - r^b = \delta^b_* - \frac{1}{\gamma}\ln\left(1 - \frac{\gamma\lambda^b(\delta^b_*)}{\lambda^{b\prime}(\delta^b_*)}\right)}
$$


### Ask Side FOC

A similar argument gives:

$$
\boxed{r^a - s = \delta^a_* - \frac{1}{\gamma}\ln\left(1 - \frac{\gamma\lambda^a(\delta^a_*)}{\lambda^{a\prime}(\delta^a_*)}\right)}
$$

Which now gives the optimal bid and ask spread implicitly

## Asymptotic Expansion in Inventory and Arrival Terms

After substituting optimal $\delta^a_*, \delta^b_*$ and assuming symmetric exponential arrival rates:

$$\lambda^a(\delta) = \lambda^b(\delta) = A e^{-k\delta}$$

We obtain:
$$
\begin{cases}
\partial_t \theta + \frac{1}{2} \sigma^2 \theta_{ss} - \frac{1}{2} \sigma^2 \gamma \theta_s^2 + \frac{A}{k+\gamma} \left( e^{-k\delta^a_*} + e^{-k\delta^b_*} \right) = 0 \\
\theta(s,q,T) = q s
\end{cases} 
$$

The HJB remains analytically intractable. The authors propose an asymptotic Taylor expansion of $\theta$ in the inventory variable $q$. 

The economic intitution of this again is likely inventory risk - it is reasonable to assume that market makers target near-zero inventory ($q\approx 0$) to minimize directional risk exposure.
$$
\theta(t,q,s) = \theta^0(s,t) + q\,\theta^1(s,t) + \frac{1}{2}q^2\,\theta^2(s,t) + \cdots \quad
$$}

Recall the bid reservation price is given by:
$$
r^b(t,s,q) = \theta(t,s,q-1) - \theta(t,s,q)
$$

We can expand the difference:
$$
\begin{aligned}
\theta(t,s,q-1) &= \theta^0 + (q-1)\theta^1 + \frac{1}{2}(q-1)^2\theta^2 + \cdots \\
&= \theta^0 + q\theta^1 - \theta^1 + \frac{1}{2}(q^2 - 2q + 1)\theta^2 + \cdots 
\end{aligned}
$$
and now subtracting to get:
$$
\begin{aligned}
r^b &= [\theta^0 + q\theta^1 - \theta^1 + \frac{1}{2}(q^2 - 2q + 1)\theta^2] - [\theta^0 + q\theta^1 + \frac{1}{2}q^2\theta^2] + \cdots \\
&= -\theta^1 - q\theta^2 + \frac{1}{2}\theta^2 + \cdots \\
&= \theta^1 + (-1 - 2q)\theta^2 + \cdots
\end{aligned}
$$

A similar argment gives us the ask price as well
$$
\begin{cases}
r^a(t,s,q) = \theta^1(s,t) + (1 - 2q)\theta^2(s,t) + \cdots \\
r^b(t,s,q) = \theta^1(s,t) + (-1 - 2q)\theta^2(s,t) + \cdots
\end{cases}
$$

Now approximate the order arrival term as well via a Taylor expansion:
$$
\frac{A}{k+\gamma} (e^{-k\delta^a} + e^{-k\delta^b}) \approx \frac{A}{k+\gamma} \left[ 2 - k(\delta^a + \delta^b) + \cdots \right]
$$

Notice that this does not depend on $q$, so if matching orders of the PDEs in $q$ gives:

$O(q)$ terms:
$$
\begin{cases}
\theta^1_t + \frac{1}{2} \sigma^2 \theta^1_{ss} = 0 \\
\theta^1(s, T) = s
\end{cases}
$$

Solution: 
$$\boxed{\theta^1(s,t) = s}$$

$O(q²)$ terms:
$$
\begin{cases}
\theta^2_t + \frac{1}{2} \sigma^2 \theta^2_{ss} - \frac{1}{2} \sigma^2 \gamma (\theta^1_s)^2 = 0 \\
\theta^2(s, T) = 0
\end{cases}
$$

**Since $\theta^1_s = 1$** this simplifies to:
$$
\boxed{\theta^2_t - \frac{1}{2} \sigma^2 \gamma = 0 \quad \to \quad \theta^2 = \frac{1}{2} \gamma \sigma^2 (T-t)}
$$

And we are finally done.

# Summary

For simplicity, we further assume the $\delta^a_*$ and $\delta^b_*$ are the same (most practical literature seem to do so).

Quotes are given by:
$$
\begin{aligned}
p^b &= s - \delta^*(t,q) \\
p^a &= s + \delta^*(t,q)
\end{aligned}
$$

and the symmetric spread is derived from the reservation bid/ask price upon substituting $\theta^1$ and $\theta^2$: 
$$
\delta^*(t,q) = \frac{\gamma \sigma^2 (T-t)}{2} + \frac{1}{\gamma} \ln\left(1 + \frac{\gamma}{k}\right)
$$

## Implementation (TBD)